# Performing speech-to-text on files

In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [154]:
from pathlib import Path
from tqdm.auto import tqdm
import pandas as pd
from IPython.display import display
import numpy as np
import os

import speech_recognition as sr
import cloudstorage as gcs

ModuleNotFoundError: No module named 'cloudstorage'

In [54]:
# configure user
user = "mitchel"
extracted_feats = True

if user.lower() == "jonas":
    BASE_PATH = Path("/users/jonvdrdo/jonas/data/aaa_contextaware/raw/uz_study/")
elif user.lower() == "mitchel":
#     BASE_PATH = Path("Z:/shares/ghep_lab/2021_VanhollebekeKappen_EEGStudy2_MIST_Cyberball_Audio/")
    BASE_PATH = Path("D:/Data/EEG_Study_2")
DATA_PATH = BASE_PATH.joinpath("Data/Raw/Audio")

In [55]:
# https://www.thepythoncode.com/article/using-speech-recognition-to-convert-speech-to-text-python
filename = "D:/Data/EEG_Study_2/Data/Raw/Audio/ppt_8_mist_speech/audio_referential_control.wav"

# initialize the recognizer
r = sr.Recognizer()

# open the file
with sr.AudioFile(filename) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    text = r.recognize_google(audio_data, language="nl-NL")
    print(text)

op de afbeeldingen afbeelding van een rekensom x door 72 keer 2 en dan 30 eten komen en dan denk ik dat ik eerst die andere twee samen heb genomen tot dat 31 uitkwam en dan kan ik zo even uit denk dat ik wel redelijk op mijn gemak was was het toch erg dat je snel snel denken maar wordt in ieder geval heerlijk rustig en de regels opgesteld dat het 72/2 met een breuk met een scherm wordt weergegeven te haakjes en dan datum 27 en 4 en dan lekker tegen en scherpe vraagteken


In [11]:
# https://www.thepythoncode.com/article/using-speech-recognition-to-convert-speech-to-text-python
filename = "D:/Data/EEG_Study_2/Data/Raw/Audio/ppt_12_mist_speech/audio_referential_control.wav"

# initialize the recognizer
r = sr.Recognizer()

# open the file
with sr.AudioFile(filename) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    text = r.recognize_google(audio_data, language="nl-BE")
    print(text)

Ik zie op het scherm en rekenopgaven staan die staat op het bovenste deel van het scherm aan de rest van het scherm is wit de rekenopgaven omvat een deling eerst en vooral die tussen haakjes staat 72/2 daarna en daarna 2 aftrekkingen en met 27 en -4 sterren is gelijk aan teken en een vraagteken waarde het antwoord zou moeten komen de rekenopgaven is een vrij moeilijke rekenopgave aan moeilijker dan ik verwacht had om hier te zien te krijgen


In [24]:
# https://textblob.readthedocs.io/en/dev/

from textblob import TextBlob
import nltk
nltk.download('punkt') # Used for tags
nltk.download('averaged_perceptron_tagger') # Used for tags
nltk.download('brown') # Used for noun phrases

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mitch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mitch\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\mitch\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.


True

In [34]:
blob = TextBlob(text)

blob.tags

blob.noun_phrases

for sentence in blob.sentences:
    print(sentence.sentiment)

Sentiment(polarity=0.0, subjectivity=0.0)


In [44]:
# Now Dutch
# https://github.com/gvisniuc/textblob-nl

from textblob_nl import PatternTagger, PatternAnalyzer

blob = TextBlob(text, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

# The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.
blob.sentiment # Polarity, subjectivity

(-0.06333333333333334, 0.4766666666666667)

In [43]:
# Word count
blob.word_counts['ik'] # For instance 'ik'

# Relative word usage (proportional):
blob.word_counts['ik']  / len(blob.words)

0.024691358024691357

# Next option needs billing, but could be helpful for confidences

In [157]:
# https://cloud.google.com/speech-to-text/docs/transcribe-client-libraries
# https://cloud.google.com/speech-to-text/docs/word-confidence#speech_transcribe_word_level_confidence_beta-python


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/mitch/OneDrive - UGent/UGent/Topics/Speech/Speech-to-text/Google-cloud/speechtextflemish01-996614be0f32.json"

from google.cloud import speech_v1p1beta1 as speech

client = speech.SpeechClient()

speech_file = "D:/Data/EEG_Study_2/Data/Raw/Audio/ppt_12_mist_speech/audio_referential_control.wav"
speech_file = "gs://speech-cybb-mist/ppt_12_mist_speech/audio_referential_control.wav"

# with open(speech_file, "rb") as audio_file:
#     content = audio_file.read()
# audio = speech.RecognitionAudio(content=content)

audio = speech.RecognitionAudio(uri=speech_file)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
#     sample_rate_hertz=16000,
    language_code="nl-BE",
    enable_word_confidence=True,
)

# response = client.recognize(config=config, audio=audio)
response = client.long_running_recognize(config=config, audio=audio)

for i, result in enumerate(response.results):
    alternative = result.alternatives[0]
    print("-" * 20)
    print("First alternative of result {}".format(i))
    print(u"Transcript: {}".format(alternative.transcript))
    print(
        u"First Word and Confidence: ({}, {})".format(
            alternative.words[0].word, alternative.words[0].confidence
        )
    )

AttributeError: 'Operation' object has no attribute 'results'

In [165]:
gcs_uri = "gs://speech-cybb-mist/ppt_12_mist_speech/audio_referential_control.wav"

client = speech.SpeechClient()

audio = speech.RecognitionAudio(uri=gcs_uri)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=44100,
    language_code="nl-BE",
)

operation = client.long_running_recognize(config=config, audio=audio)

print("Waiting for operation to complete...")
response = operation.result(timeout=90)

# Each result is for a consecutive portion of the audio. Iterate through
# them to get the transcripts for the entire audio file.
for result in response.results:
    # The first alternative is the most likely one for this portion.
    print(u"Transcript: {}".format(result.alternatives[0].transcript))
    print("Confidence: {}".format(result.alternatives[0].confidence))

Waiting for operation to complete...
Transcript: Ik zie op het scherm en rekenopgaven staan die staat op het bovenste deel van het scherm aan van het scherm is wit de rekenopgaven omvat een deling eerst en vooral die tussen haakjes staat 72/2 daarna en daarna 2 aftrekkingen en 27 en -4 en die gisteren gelijk aanteken een vraagteken waarde het antwoord zou moeten komen de rekenopgaven is een vrij
Confidence: 0.9059465527534485
Transcript: moeilijke rekenopgave aan moeilijker dan ik verwacht had om hier te zien te krijgen
Confidence: 0.9562501311302185


In [167]:
gcs_uri = "gs://speech-cybb-mist/ppt_21_mist_speech/audio_referential_control.wav"

client = speech.SpeechClient()

audio = speech.RecognitionAudio(uri=gcs_uri)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=44100,
    language_code="nl-BE",
)

operation = client.long_running_recognize(config=config, audio=audio)

print("Waiting for operation to complete...")
response = operation.result(timeout=90)

# Each result is for a consecutive portion of the audio. Iterate through
# them to get the transcripts for the entire audio file.
for result in response.results:
    # The first alternative is the most likely one for this portion.
    print(u"Transcript: {}".format(result.alternatives[0].transcript))
    print("Confidence: {}".format(result.alternatives[0].confidence))

Waiting for operation to complete...
Transcript: om de afbeelding is een rekensom zien met hand helemaal in welk jaar is tussenhaakjes 50 gedeeld door 2 dan daarna volgt een intake en dan 21 - 4 = vraagteken en rekenen 50 gedeeld door 3236 min 27
Confidence: 0.8086049556732178
Transcript: Don
Confidence: 0.5828940868377686
Transcript:  natuurlijk 15 uur
Confidence: 0.5208269953727722
Transcript:  waarschijnlijk
Confidence: 0.9562501311302185
Transcript:  rekensom natuurlijk niet opgehaald Ik noem jou dat natuurlijk op volgorde van bewerking die dingen moet doen en dan
Confidence: 0.7964165806770325


# The quest to confidence

In [128]:
r = sr.Recognizer()
with sr.WavFile(filename) as source:
    audio = r.record(source)
list = r.recognize_google(audio, None, "nl-BE", True)

In [129]:
list

{'alternative': [{'transcript': 'op de afbeeldingen afbeelding van een rekensom x door 72 keer 2 en dan 30 eten komen en dan denk ik dat ik eerst die andere twee samen heb genomen tot dat 31 uitkwam en dan kan ik zo even uit denk dat ik wel redelijk op mijn gemak was was het toch erg dat je snel snel denken maar wordt in ieder geval heerlijk rustig en de regels opgesteld dat het 72/2 met een breuk met een scherm wordt weergegeven te haakjes en dan datum 27 en 4 en dan lekker tegen en scherpe vraagteken',
   'confidence': 0.91912794},
  {'transcript': 'op de afbeeldingen afbeelding van een rekensom x door 72 keer 2 en dan 30 te komen en dan denk ik dat ik eerst die andere twee samen heb genomen tot dat 31 uitkwam en dan kan ik zo even uit denk dat ik wel redelijk op mijn gemak was was het toch erg dat je snel snel denken maar wordt in ieder geval heerlijk rustig en de regels opgesteld dat het 72/2 met een breuk met het scherm wordt weergegeven te haakjes en dan datum 27 en 4 en dan lekk

In [127]:
help(r.recognize_google)

Help on method recognize_google in module speech_recognition:

recognize_google(audio_data, key=None, language='en-US', show_all=False) method of speech_recognition.Recognizer instance
    Performs speech recognition on ``audio_data`` (an ``AudioData`` instance), using the Google Speech Recognition API.
    
    The Google Speech Recognition API key is specified by ``key``. If not specified, it uses a generic key that works out of the box. This should generally be used for personal or testing purposes only, as it **may be revoked by Google at any time**.
    
    To obtain your own API key, simply following the steps on the `API Keys <http://www.chromium.org/developers/how-tos/api-keys>`__ page at the Chromium Developers site. In the Google Developers Console, Google Speech Recognition is listed as "Speech API".
    
    The recognition language is determined by ``language``, an RFC5646 language tag like ``"en-US"`` (US English) or ``"fr-FR"`` (International French), defaulting to US E

In [138]:
# Check differences between options
import difflib

for a,b in cases:     
    print('{} => {}'.format(a,b))  
    for i,s in enumerate(difflib.ndiff(a, b)):
        if s[0]==' ': continue
        elif s[0]=='-':
            print(u'Delete "{}" from position {}'.format(s[-1],i))
        elif s[0]=='+':
            print(u'Add "{}" to position {}'.format(s[-1],i))    
    print() 

op de afbeeldingen afbeelding van een rekensom x door 72 keer 2 en dan 30 eten komen en dan denk ik dat ik eerst die andere twee samen heb genomen tot dat 31 uitkwam en dan kan ik zo even uit denk dat ik wel redelijk op mijn gemak was was het toch erg dat je snel snel denken maar wordt in ieder geval heerlijk rustig en de regels opgesteld dat het 72/2 met een breuk met een scherm wordt weergegeven te haakjes en dan datum 27 en 4 en dan lekker tegen en scherpe vraagteken => op de afbeeldingen afbeelding van een rekensom x door 72 keer 2 en dan 30 uit te komen en dan denk ik dat ik eerst die andere twee samen heb genomen tot dat 31 uitkwam en dan kan ik zo even uit denk dat ik wel redelijk op mijn gemak was was het toch erg dat je snel snel denken maar wordt in ieder geval heerlijk rustig en de regels opgesteld dat het 72/2 met een breuk met het scherm wordt weergegeven te haakjes en dan datum 27 en 4 en dan lekker tegen en scherpe vraagteken
Delete "e" from position 74
Add "u" to positi

In [148]:
for i, s in enumerate(difflib.ndiff(a, b)):
    if s[0]==' ': continue
    elif s[0]=='-':
        print(u'Delete "{}" from position {}'.format(s[-1],i))
    elif s[0]=='+':
        print(u'Add "{}" to position {}'.format(s[-1],i))   


Delete "e" from position 74
Add "u" to position 75
Add "i" to position 76
Add " " to position 78
Add "t" to position 79
Delete "n" from position 81
Add "h" to position 376
Add "t" to position 378
Delete "e" from position 379
Delete "n" from position 380


In [144]:
s

'  n'